In [9]:
def getpayload(msg):
    return __getpayload_rec__(msg, payloadresult="")


def __getpayload_rec__(msg, payloadresult):
    payload = msg.get_payload()
#     print(payload)
    if str(msg.get('content-transfer-encoding')).lower() == "base64":
        
        payload = msg.get_payload(decode=True)
#         print("entered here:", payload)
#         print(type(payload))

    if payload and msg.is_multipart():
        for subMsg in payload:
            payloadresult += __getpayload_rec__(subMsg, payloadresult)
    else:
        if type(payload) == bytes:
#             print("AM BYTES")
            payload_asbytes = str(payload)
#             try:

#             except (UnicodeDecodeError, AttributeError):
#                 print("Found Error!")
#                 pass
        
            return msg.get_content_type() + "\t" + payload_asbytes + "\n"
        
        elif type(payload) == str:
            return msg.get_content_type() + "\t" + payload + "\n"
        elif type(payload) == None:
            print("nonetype in getpayload")
    return payloadresult

def getpayload_dict(msg):
    return __getpayload_dict_rec__(msg, [])


def __getpayload_dict_rec__(msg, payloadresult):
    payload = msg.get_payload()
    if msg.is_multipart():
        for subMsg in payload:
            __getpayload_dict_rec__(subMsg, payloadresult)
    else:
        payloadresult.append({"mimeType": msg.get_content_type(), "payload": payload})
    return payloadresult

In [ ]:
import time
from itertools import chain
import email
import imaplib
import pprint as pp

import argparse


# get commandline arguments
# parser = argparse.ArgumentParser()
# parser.add_argument('-p', '--password')
# parser.add_argument('-u', '--username')
# args = parser.parse_args()

# print("Command line arguments:");print(args);print(78*'-')


imap_ssl_host = 'imap.gmail.com'  # imap.mail.yahoo.com
imap_ssl_port = 993
username = 'phishingdetective@gmail.com'
password = 'phishtooldetect97'

# Restrict mail search. Be very specific.
# Machine should be very selective to receive messages.
criteria = {
  # 'FROM':    'isaacpitblado@tamu.edu',
  # 'SUBJECT': 'PHISHING',
  # 'BODY':    'SECRET SIGNATURE',
}
uid_max = 0


def search_string(uid_max, criteria):
  c = list(map(lambda t: (t[0], '"'+str(t[1])+'"'), criteria.items())) + [('UID', '%d:*' % (uid_max+1))]
  return '(%s)' % ' '.join(chain(*c))
  # Produce search string in IMAP format:
  #   e.g. (FROM "me@gmail.com" SUBJECT "abcde" BODY "123456789" UID 9999:*)


def get_first_text_block(msg):
  type = msg.get_content_maintype()

  if type == 'multipart':
    for part in msg.get_payload():
      if part.get_content_maintype() == 'text':
        return part.get_payload()
  elif type == 'text':
    return msg.get_payload()


server = imaplib.IMAP4_SSL(imap_ssl_host, imap_ssl_port)
server.login(username, password)
server.select('INBOX')

# result, data = server.uid('search', None, search_string(uid_max, criteria))
result, data = server.search(None, 'ALL')

uids = [int(s) for s in data[0].split()]
# print("original uids: ", uids)
if uids:
  uid_max = max(uids)
  # Initialize `uid_max`. Any UID less than or equal to `uid_max` will be ignored subsequently.

server.logout()


# Keep checking messages ...
# I don't like using IDLE because Yahoo does not support it.
while 1:
  # Have to login/logout each time because that's the only way to get fresh results.

  server = imaplib.IMAP4_SSL(imap_ssl_host, imap_ssl_port)
  server.login(username, password)
  server.select('INBOX')

  # result, data = server.uid('search', None, search_string(uid_max, criteria))
  result, data = server.search(None, 'ALL')

  uids = [int(s) for s in data[0].split()]
  # print('UID: ', uids)
  # for uid in uids:
  for i in range(len(uids) -1, len(uids) - 2, -1):
    # Have to check again because Gmail sometimes does not obey UID criterion.
    uid = uids[i]
    # if uid > uid_max:
    if True:
      print("uid: ", uid)
      result, data = server.uid('fetch', str(uid), '(RFC822)')  # fetch entire message
      # print("res: ", result)
      # print("data: ", data)
      # print("data:")
      # pp.pprint(data)
      


      for response_part in data:
        if isinstance(response_part, tuple):
          # print("response_part: ", response_part)
          # print("response_part[1]: ", response_part[1])
          msg = email.message_from_string(response_part[1].decode('utf-8'))
          print('msg_keys: ');pp.pprint(msg.keys())
          print("msg: \n", msg)
          
          email_subject = msg['subject']
          email_from = msg['from']
          email_payload = msg.get_payload(decode=True)
          print('From : ' + email_from + '\n')
          print('Subject : ' + email_subject + '\n')
          print('Payload :')
          print(email_payload)
          print('Content-type', msg['Content-type'])
          print('Mime-Version', msg['Mime-Version'])
          print("is multi part: ", msg.is_multipart())
          if msg.is_multipart():
            for payload in msg.get_payload():
              print('To:\t\t', msg['To'])
              print('From:\t',     msg['From'])
              print('Subject:', msg['Subject'])
              print('Date:\t',msg['Date'])
              for part in msg.walk():
                print("*" * 78)
                print("part.content_type: {0}".format(part.get_content_type()))
                # if (part.get_content_type() == 'text/plain') and (part.get('Content-Disposition') is None):
                print('Body:\t',part.get_payload())
              
              
              # break


          # for part in msg.walk():
          #   print("part: ", part)
            
          
          

      
      # msg = email.message_from_string(data[0][1])
      
      # uid_max = uid
    
      # text = get_first_text_block(msg)
      # print('New message :::::::::::::::::::::')
      # print(text)

      server.logout()
      break
      time.sleep(1)


In [12]:
import time
from itertools import chain
import email
import imaplib
import pprint as pp
import email
import email.mime.message
import base64
# import mailparser

# mail = mailparser.parse_from_file(f)
# mail = mailparser.parse_from_file_obj(fp)
# mail = mailparser.parse_from_string(raw_mail)
# mail = mailparser.parse_from_bytes(byte_mail)

import argparse


# get commandline arguments
# parser = argparse.ArgumentParser()
# parser.add_argument('-p', '--password')
# parser.add_argument('-u', '--username')
# args = parser.parse_args()

# print("Command line arguments:");print(args);print(78*'-')


imap_ssl_host = 'imap.gmail.com'  # imap.mail.yahoo.com
imap_ssl_port = 993
username = 'phishingdetective@gmail.com'
password = 'phishtooldetect97'

# Restrict mail search. Be very specific.
# Machine should be very selective to receive messages.
criteria = {
  # 'FROM':    'isaacpitblado@tamu.edu',
  # 'SUBJECT': 'PHISHING',
  # 'BODY':    'SECRET SIGNATURE',
}
uid_max = 0


def search_string(uid_max, criteria):
  c = list(map(lambda t: (t[0], '"'+str(t[1])+'"'), criteria.items())) + [('UID', '%d:*' % (uid_max+1))]
  return '(%s)' % ' '.join(chain(*c))
  # Produce search string in IMAP format:
  #   e.g. (FROM "me@gmail.com" SUBJECT "abcde" BODY "123456789" UID 9999:*)


def get_first_text_block(msg):
  type = msg.get_content_maintype()

  if type == 'multipart':
    for part in msg.get_payload():
      if part.get_content_maintype() == 'text':
        return part.get_payload()
  elif type == 'text':
    return msg.get_payload()


server = imaplib.IMAP4_SSL(imap_ssl_host, imap_ssl_port)
server.login(username, password)
server.select('INBOX')

# result, data = server.uid('search', None, search_string(uid_max, criteria))
result, data = server.search(None, 'ALL')

uids = [int(s) for s in data[0].split()]
# print("original uids: ", uids)
if uids:
#   uid_max = max(uids) # FIXME
    uid_max = 6
  # Initialize `uid_max`. Any UID less than or equal to `uid_max` will be ignored subsequently.

server.logout()


# Keep checking messages ...
# I don't like using IDLE because Yahoo does not support it.
while 1:
    # Have to login/logout each time because that's the only way to get fresh results.

    server = imaplib.IMAP4_SSL(imap_ssl_host, imap_ssl_port)
    server.login(username, password)
    server.select('INBOX')

    # result, data = server.uid('search', None, search_string(uid_max, criteria))
    result, data = server.search(None, 'ALL')

    uids = [int(s) for s in data[0].split()]
    print('UID: ', uids)
    for uid in uids:
    #   for i in range(len(uids) -1, len(uids) - 2, -1):
    # Have to check again because Gmail sometimes does not obey UID criterion.
    #     uid = uids[i]
        if uid > uid_max:
        #     if True:
            print("uid: ", uid)
            result, data = server.uid('fetch', str(uid), '(RFC822)')  # fetch entire message
            
            # print("res: ", result)
            # print("data: ", data)
            # print("data:")
            # pp.pprint(data)

            print("setting new uid_max: ", uid)
            uid_max = uid
            
#             print("data:", data)
            
            attachment_result = []

            for response_part in data:
                if isinstance(response_part, tuple):
                    # print("response_part: ", response_part)
                    # print("response_part[1]: ", response_part[1])
                    msg = email.message_from_string(response_part[1].decode('utf-8'))
#                     msg = email.message_from_string(response_part[1].decode('utf-8'))
                    print(msg)

                    for part in msg.walk():

                        if part['Content-Type'] and part['Content-Type'].find("message/rfc822") != -1:

#                             attachment = part.get_payload()
                            print("PAYLOAD LEN: ", len(attachment))
                            for i in range(len(attachment)):
#                                 print("payload i: ",i, attachment[i].as_string())
#                                 print("keys: ", attachment[i].keys())
#                                 print("type: ", type(attachment[i]))
                                decoded = email.message_from_bytes(base64.b64decode(attachment[i].as_string()))
#                                 decoded
                                attached_message = getpayload_dict(decoded)
    
                                totalsize = 0
                                payload = getpayload_dict(attached_message)
                                for part in payload:
                                    totalsize += len(re.sub(r'\s+','',part["payload"]))


                                for finder in myfinders:
                                    tmpDict[finder.getFeatureTitle()] = finder.getFeature(msg)

                                tmpDict["Phishy"] = phishy
#                                 tmpDict["id"] = i


                                for finder in myfinders:
                                    tmpDict[finder.getFeatureTitle()] = finder.getFeature(msg)

                    #             pp.pprint(tmpDict)
                                data.append(tmpDict)

                                i+=1
                                if stop and i >= stop:
                                    break
#                                 for elem in decoded.walk():
#                                     print(elem)
            
                                
                        else:
                            print("not message/rfc822: ", part['Content-Type'])
                        
                        
                df = data
                print(attachment_result)
#   server.logout()
#   break

    time.sleep(1)


UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
uid:  7
setting new uid_max:  7
Delivered-To: phishingdetective@gmail.com
Received: by 2002:a05:6000:1052:0:0:0:0 with SMTP id c18csp2116566wrx;
        Sun, 1 Dec 2019 13:44:30 -0800 (PST)
X-Received: by 2002:a92:5805:: with SMTP id m5mr14026628ilb.59.1575236670844;
        Sun, 01 Dec 2019 13:44:30 -0800 (PST)
ARC-Seal: i=1; a=rsa-sha256; t=1575236670; cv=none;
        d=google.com; s=arc-20160816;
        b=ac+6+iMvT82iP3R6o9IP7/xwwPHpV6I0AjzMoAI/0KNNVkdimZzcUL7zksHCGN5j+5
         QRkHGvQN6f1/WTqbvzLozxYLzEfPwg/RgUDq/dIBjQpm/IEuFo2pcXcZj/srlPm7bnJz
         tVzpXAPs4syRmpo4uWlsa7WviCI7VxDsOP5STQq3lVusi2hlWGr17V5eebBdcDrqexSB
         mrTnhlWPDqOa+At/dp59rZv4o/C1yik3bBvckAwkLh7iuPeCVgO51n9/UcT+rW8iF0fG
         sll8dDdR0kRdGl41dxQbUGq0MzOdx/4/6Bsi0OoQL9ruMqEC3FkHC5SGQ9Vrq0C8xEaX
         G3JQ==
ARC-Message-Signature: i=1; a=rsa-sha256; c=relaxed/relaxed; d=google.com; s=arc-20160816;
        h=to:subject:message-id:date:from:mime-version:dkim-signat

setting new uid_max:  9
Delivered-To: phishingdetective@gmail.com
Received: by 2002:a05:6000:1052:0:0:0:0 with SMTP id c18csp2250341wrx;
        Sun, 1 Dec 2019 16:46:30 -0800 (PST)
X-Received: by 2002:a92:a189:: with SMTP id b9mr67877062ill.259.1575247590551;
        Sun, 01 Dec 2019 16:46:30 -0800 (PST)
ARC-Seal: i=1; a=rsa-sha256; t=1575247590; cv=none;
        d=google.com; s=arc-20160816;
        b=XunnlYZKcYryVOCsGoyyxr37yBChYQ8Y2BqOs35LqWjBrOZ+73VpZyq88gJsWNHBu6
         EHL5x9Iurv5sBBaPIfJ8+Orm/wulSXr4x/XtlG6OJqMPn+ttJzYAperroM4bz8cqrUSx
         CCFZUzKd982QaEnzlNGcEoBdfvQ8+Miquj6Hn0vtBTzYieTMuqymPuQwluM7sLLEyKnu
         x+zB6b0vI9DlyETYBCJPyiWmgXUpYGmXBexnTIXpSMPcO7Is9Qzn/pq4z6cEfCJg1jxu
         CEAxsuGNvkL7rMV3iOT+l2RuPNxFBdBX7za/0Wx/BC+UY5wMttO+CtkYRFS60AY3fARu
         hzIQ==
ARC-Message-Signature: i=1; a=rsa-sha256; c=relaxed/relaxed; d=google.com; s=arc-20160816;
        h=to:subject:message-id:date:from:mime-version:dkim-signature;
        bh=zPP6KEQeRozz6VDCa1byW3q4O

UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9]


KeyboardInterrupt: 